In [15]:
import os
import re
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm
from scipy.stats import mode
from scipy.ndimage import median_filter
from functools import partial

In [16]:
# --- 설정 ---
wav_paths = [
    "unwelcomeSchool/vocals.wav",
    "unwelcomeSchool/bass.wav",
    "unwelcomeSchool/drums.wav",
    "unwelcomeSchool/other.wav"
]

# 우선순위 설정
preferred_order = [3, 2, 1, 0]
# 전체 에너지값 중 5%미만일 시 자르기 위함.
energy_threshold_ratio = 0.05

osu_input = "UnwelcomeSchool_Hard.txt"
base_name = os.path.splitext(os.path.basename(osu_input))[0]

# 트랙 증폭
amplify_indices = [3]
amplify_factor = 1.5
default_clip_ms = 333
# 앞 뒤, FadeIn, Out
fade_ms = 50

In [17]:
# --- WAV 로드 ---
waves, sr = [], None
for idx, p in enumerate(wav_paths):
    y, sr_tmp = librosa.load(p, sr=None)
    if sr is None:
        sr = sr_tmp
    elif sr != sr_tmp:
        raise ValueError("샘플링 레이트 불일치")
    if idx in amplify_indices:
        y = np.clip(y * amplify_factor, -1.0, 1.0)
    waves.append(y)

In [18]:
# --- osu 파일 파싱 ---
with open(osu_input, 'r', encoding='utf-8') as f:
    osu_text = f.read()

hit_match = re.search(r"\[HitObjects\](.*)$", osu_text, re.S)
if not hit_match:
    raise ValueError("[HitObjects] 섹션을 찾을 수 없습니다.")
hit_lines = [l for l in hit_match.group(1).strip().splitlines() if l.strip()]
times_ms = [int(line.split(',')[2]) for line in hit_lines]

events_match = re.search(r"\[Events\](.*?)(?=\n\[|$)", osu_text, re.S)

In [19]:
"""
기존 에너지만 고려한 방식
"""
def smooth_none(indices):
    return indices

"""
mode 필터 사용 Window-based Majority Voting
Window 만들어서 가장 많이 나오는 값 선택.
"""
def smooth_mode(indices, window=5):
    pad = window // 2
    padded = np.pad(indices, (pad, pad), mode='edge')
    return [int(mode(padded[i:i+window], keepdims=False).mode) for i in range(len(indices))]

"""
Median filter 사용해서 OutLier 제거.
"""
def smooth_median(indices, size=5):
    return median_filter(indices, size=size).tolist()

"""
우선순위 정해서 해당 부분에 대해 입계값보다 높으면 순서대로 노트에 할당
"""
def smooth_priority(dominant_indices, *, times_ms, sr, waves, preferred_order, threshold=0.3):
    time_to_indices = {}
    for i, t in enumerate(times_ms):
        time_to_indices.setdefault(t, []).append(i)

    dominant_by_index = [None] * len(times_ms)

    for t, indices in time_to_indices.items():
        start = int(sr * t / 1000)
        end = start + int(sr * default_clip_ms / 1000)
        end = min(end, len(waves[0]))
        energies = [np.sum(w[start:end] ** 2) for w in waves]
        total_energy = sum(energies)

        if len(indices) > 1:
            sorted_idx = sorted(range(len(energies)), key=lambda x: -energies[x])
            for j, i in enumerate(indices):
                dominant_by_index[i] = sorted_idx[j % len(waves)]
        else:
            chosen = None
            for p in preferred_order:
                if energies[p] >= threshold * total_energy:
                    chosen = p
                    break
            if chosen is None:
                chosen = int(np.argmax(energies))
            dominant_by_index[indices[0]] = chosen

    return dominant_by_index


In [20]:
smoothing_methods = {
    "base": smooth_none,
    "mode": smooth_mode,
    "median": smooth_median,
    "priority": partial(
        smooth_priority,
        times_ms=times_ms,
        sr=sr,
        waves=waves,
        preferred_order=preferred_order,
        threshold=energy_threshold_ratio
    )
}

In [21]:
# --- base dominant index 계산 ---
base_dominant_indices = []
for i, time_ms in enumerate(times_ms):
    start = int(sr * time_ms / 1000)
    if i < len(times_ms) - 1:
        next_ms = times_ms[i+1]
        clip_ms = min(default_clip_ms, next_ms - time_ms)
    else:
        clip_ms = default_clip_ms
    end = min(start + int(sr * clip_ms / 1000), len(waves[0]))
    energies = [np.sum(w[start:end]**2) for w in waves]
    idx = int(np.argmax(energies))
    base_dominant_indices.append(idx)

In [22]:
# --- 노트 추출 함수 ---
def extract_clips(strategy_name, dominant_indices):
    print(f"\n▶ 추출 중: {strategy_name} 방식")
    output_dir = f"splited_notes_{strategy_name}"
    os.makedirs(output_dir, exist_ok=True)
    osu_output = f"{base_name}_{strategy_name}.osu"

    fade_len = int(sr * fade_ms / 1000)
    ramp = np.linspace(0, 1, fade_len)

    silent_tracks = [w.copy() for w in waves]
    new_events, new_hit_lines = [], []
    counter = 1

    for i, line in tqdm(enumerate(hit_lines), total=len(hit_lines), desc=f"Extracting ({strategy_name})"):
        parts = line.split(',')
        time_ms = times_ms[i]
        if i < len(times_ms) - 1:
            next_ms = times_ms[i+1]
            clip_ms = min(default_clip_ms, next_ms - time_ms)
        else:
            clip_ms = default_clip_ms
        clip_len = int(sr * clip_ms / 1000)

        idx = dominant_indices[i]
        start = int(sr * time_ms / 1000)
        end = min(start + clip_len, len(waves[0]))

        clip = waves[idx][start:end].copy()
        if len(clip) > 2 * fade_len:
            clip[:fade_len] *= ramp
            clip[-fade_len:] *= ramp[::-1]
        else:
            clip *= np.linspace(0, 1, len(clip))

        silent_tracks[idx][start:end] = 0

        clip_name = f"note_{idx+1}_{counter}.wav"
        sf.write(os.path.join(output_dir, clip_name), clip, sr)

        tail = parts[-1]
        effects, _ = (tail.rsplit(':', 1) if ':' in tail else (tail, ''))
        parts[-1] = f"{effects}:{clip_name}"
        new_hit_lines.append(','.join(parts))
        new_events.append(f"Sample,{time_ms},0,\"{clip_name}\",50")
        counter += 1

    # accompaniment 저장
    accompaniment = np.zeros_like(waves[0])
    for tr in silent_tracks:
        accompaniment += tr
    sf.write(os.path.join(output_dir, "audio.wav"), accompaniment, sr)

    # osu 파일 저장
    header = osu_text[:events_match.start()]
    between = osu_text[events_match.end():hit_match.start()]
    footer = osu_text[hit_match.end():]
    with open(osu_output, 'w', encoding='utf-8') as f:
        f.write(header)
        f.write("[Events]\n")
        for ev in new_events:
            f.write(ev + "\n")
        f.write("\n" + between)
        f.write("[HitObjects]\n")
        for hl in new_hit_lines:
            f.write(hl + "\n")
        f.write(footer)

    print(f"{strategy_name} 완료 - 파일 위치: {output_dir}, {osu_output}")

In [23]:
# --- smoothing 방식 적용 및 노트 추출 ---
for name, func in smoothing_methods.items():
    smoothed_indices = func(base_dominant_indices)
    extract_clips(name, smoothed_indices)


▶ 추출 중: base 방식


Extracting (base): 100%|██████████████████████████████████████████████████████████| 1338/1338 [00:04<00:00, 321.99it/s]


base 완료 - 파일 위치: splited_notes_base, UnwelcomeSchool_Hard_base.osu

▶ 추출 중: mode 방식


Extracting (mode): 100%|██████████████████████████████████████████████████████████| 1338/1338 [00:04<00:00, 331.82it/s]


mode 완료 - 파일 위치: splited_notes_mode, UnwelcomeSchool_Hard_mode.osu

▶ 추출 중: median 방식


Extracting (median): 100%|████████████████████████████████████████████████████████| 1338/1338 [00:04<00:00, 332.33it/s]


median 완료 - 파일 위치: splited_notes_median, UnwelcomeSchool_Hard_median.osu

▶ 추출 중: priority 방식


Extracting (priority): 100%|██████████████████████████████████████████████████████| 1338/1338 [00:05<00:00, 224.19it/s]


priority 완료 - 파일 위치: splited_notes_priority, UnwelcomeSchool_Hard_priority.osu
